In [1]:
import pandas as pd
!pip install -q wordcloud
import wordcloud
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [2]:
df = pd.read_csv('/content/Review_data.csv')
df

,review,rating
0,It was nice produt. I like it's design a lot. ...,5
1,awesome sound....very pretty to see this nd th...,5
2,awesome sound quality. pros 7-8 hrs of battery...,4
3,I think it is such a good product not only as ...,5
4,awesome bass sound quality very good bettary l...,5
...,...,...
9971,GoodREAD MORE,5
9972,Everything is amazimg but the built is very li...,5
9973,GoodREAD MORE,5
9974,Best headphone i have ever used....READ MORE,5


In [3]:
df['rating'] = df['rating'].replace(5,'Good')
df['rating'] = df['rating'].replace(4,'Good')
df['rating'] = df['rating'].replace(3,'Good')
df['rating'] = df['rating'].replace(2,'Bad')
df['rating'] = df['rating'].replace(1,'Bad')

In [4]:
#preprocess data
df['review'] = df['review'].str.lower()  #Convert to lower case
df['review'] = df['review'].str.replace('\d+', '') #removing numbers
df['review'] = df['review'].str.replace('.','')
df['review'] = df['review'].str.replace('-','')
df['review'] = df['review'].str.replace('?','')
df['review'] = df['review'].str.replace("'",'')
df['review'] = df['review'].str.replace('  ','')  #for extra spaces


In [5]:
#tokenization and stemming
from nltk.stem.snowball import SnowballStemmer
df['tokenized_sents'] = df.apply(lambda row: nltk.word_tokenize(row['review']), axis=1)
stemmer = SnowballStemmer("english")
df['stemmed'] = df['tokenized_sents'].apply(lambda x: [stemmer.stem(y) for y in x]) # Stem every word.
df = df.drop(columns=['tokenized_sents'])
df['review'] = df['stemmed'].apply(lambda x: ' '.join(map(str, x)))
df = df.drop(columns=['stemmed'])

In [6]:
x = df.iloc[:,0]
y = df.iloc[:,1]

In [7]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=0,stratify=y)

In [8]:
#Vectorization and model fitting using pipeline
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
text_model = Pipeline([('tfidi',TfidfVectorizer()),('model',SVC())])
text_model.fit(x_train,y_train)

Pipeline(memory=None,
         steps=[('tfidi',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('model',
                 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', d

In [9]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
y_pred = text_model.predict(x_test)
accuracy_score(y_test,y_pred)

0.9370489174017642

In [10]:
#confusion matrix
confusion_matrix(y_test,y_pred)

array([[ 117,  133],
       [  24, 2220]])

In [11]:
#Classification report
classification_report(y_test,y_pred)

'              precision    recall  f1-score   support\n\n         Bad       0.83      0.47      0.60       250\n        Good       0.94      0.99      0.97      2244\n\n    accuracy                           0.94      2494\n   macro avg       0.89      0.73      0.78      2494\nweighted avg       0.93      0.94      0.93      2494\n'

In [12]:
import joblib
joblib.dump(text_model,'Email_Class')

['Email_Class']

In [13]:
!pip install streamlit --quiet
!pip install pyngrok==4.1.1 --quiet
from pyngrok import ngrok

     |████████████████████████████████| 8.2MB 7.6MB/s 
     |████████████████████████████████| 174kB 55.2MB/s 
     |████████████████████████████████| 81kB 9.4MB/s 
     |████████████████████████████████| 4.2MB 44.0MB/s 
     |████████████████████████████████| 112kB 37.9MB/s 
     |████████████████████████████████| 71kB 8.0MB/s 
     |████████████████████████████████| 122kB 54.0MB/s 
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.5.5 which is incompatible.


In [14]:
%%writefile app.py
import streamlit as st
import sklearn
import joblib
model =joblib.load('Email_Class')

add_selectbox = st.sidebar.radio("Our page",['Home',"Rate us"])
if(add_selectbox == 'Home'):
  st.title("Review classifier")
  st.image('https://static.businessworld.in/article/article_extra_large_image/1600858456_HulSrV_Flipkart.jpg')

  ip = st.text_input('Enter your review')
  p = model.predict([ip])
  if st.button('Predict'):
    st.header(op[0])
    if(op[0]=='Good'):
      st.markdown(""":smile:""")
    else:
      st.markdown(""":angry:""")
else:
  st.title("Rate US")
  st.slider("Rating",0,5)
  if st.button("Submit"):
    st.header("Thanks for the rating")
  
  

Writing app.py


In [15]:
!nohup streamlit run app.py&
url = ngrok.connect(port = '8501')
url

nohup: appending output to 'nohup.out'


'http://cf54685b0403.ngrok.io'

In [16]:
!pip install pipreqs

In [17]:
!pipreqs /content

INFO: Successfully saved requirements file in /content/requirements.txt
